Optimization code of a 3-bus version of integrated water power system (WaPS) of the paper [Optimal Operation of Integrated Water–Power Systems Under Contingencies](https://ieeexplore.ieee.org/document/9760131/) using Pyomo


*   Elmira Farahani
*   Department of Energy Engineering
*   Sharif University of Technology
*   gh.farahani.elmira@gmail.com



# Import libraries

In [1]:
!pip install pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.0 MB/s eta 0:00:00


In [2]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 120875 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

In [3]:
import pyomo.environ as pyo
import matplotlib.pyplot as plt
from pyomo.environ import *
import numpy as np

# Define model

In [4]:
model = pyo.ConcreteModel()

# Define variables & parameters

In [5]:
time_idx = [i for i in range(24)]
nodes_number = 7                                                                        # No. of nodes in simplified water system
breakpoints_number = 5                                                                  # No. of linearization breakpoints
bus_number = 3
breakpoints = [i for i in range(breakpoints_number)]
two_units_idx = [(i,j) for i in range(2) for j in range(24)]                             # index list of 2 unit system-pump
pipelines = {0:(1,2),1:(2,3),2:(3,4),3:(5,6),4:(6,7)}                                   # pipeline 1 = connection between nodes 1 and 2
translines = {0:(1,2),1:(1,3),2:(3,2)}                          # transmission lines...line 1 = connection of bus 1 to bus 2
Q_pipe_idx = [(j,t) for j in pipelines.keys() for t in time_idx]
Y_p_head_idx = [(i,j,t) for i in breakpoints for j in pipelines.keys() for t in time_idx]
H_pipe_idx = [(j,n,t) for j in pipelines.keys() for n in range(nodes_number) for t in time_idx]
H_pump_idx = [(i,n,t) for i in range(2) for n in range(nodes_number) for t in time_idx]
X_pipe_idx = [(i,j,t) for i in range(breakpoints_number) for j in pipelines.keys() for t in time_idx]
X_pump_idx = [(u,j,t) for u in range(breakpoints_number) for j in range(2) for t in time_idx]
transline_idx = [(k,t) for k in translines.keys() for t in time_idx]
Voltage_angle_idx = [(i,j) for i in range(bus_number) for j in time_idx]                # index list of 2 unit system-Voltage angle
generating_units = [g for g in range(2)]
q_p_index = [(u,i) for u in range(breakpoints_number) for i in range(2)]
q_j_index = [(u,j) for u in range(breakpoints_number) for j in pipelines.keys()]
Pipe_param = [0.15, 0.15, 0.3, 0.25, 0.15]
H_min = 20            # min nodal pressure heads
H_max = 1000          # max nodal pressure heads
Q_max = 16200                  # max/min water flow rate to the network
a = [[178,95,2],[178,42,2]]    # Performance parameters of pumps a
b = [[295, 134],[295, 134]]    # Performance parameters of pumps b
Water_demand_node4 = {1:143.856, 2:92.484 , 3:82.1988 , 4:92.484 , 5:113.0292 , 6:256.8852 , 7:534.3228 , 8:688.4532 , 9:719.28 , 10:698.724 , 11:647.352 , 12:575.424 , 13:503.496 , 14:462.3948 , 15:421.2972 , 16:431.568 , 17:462.3948 , 18:524.052 , 19:565.1532 , 20:565.1532 , 21:524.052 , 22:482.94 , 23:400.7412 , 24:277.4412 }
Water_demand_node7 = {1:179.856, 2:128.484 , 3:118.1988 , 4:128.484 , 5:149.0292 , 6:292.8852 , 7:570.3228 , 8:724.4532 , 9:755.28 , 10:734.724 , 11:683.352 , 12:611.424 , 13:539.496 , 14:498.3948 , 15:457.2972 , 16:467.568 , 17:498.3948 , 18:560.052 , 19:601.1532 , 20:601.1532 , 21:560.052 , 22:518.94 , 23:436.7412 , 24:313.4412 }
sys_elec_demand = {1:1775.835, 2:1669.815 , 3:1590.3 , 4:1563.795 , 5:1563.795 , 6:1590.3 , 7:1961.37 , 8:2279.43 , 9:2517.975 , 10:2544.48 , 11:2544.48 , 12:2517.975 , 13:2517.975 , 14:2517.975 , 15:2464.965 , 16:2464.965 , 17:2623.995 , 18:2650.5 , 19:2650.5 , 20:2544.48 , 21:2411.955 , 22:2199.915 , 23:1934.865 , 24:1669.815 }

X_idx = [(p,t,u,m) for p in range(2) for t in time_idx for u in range(breakpoints_number) for m in range(breakpoints_number)]
Q_p_max = 1000
W_t_max = 410
q = np.zeros((2,breakpoints_number))
w = np.zeros((2,breakpoints_number))
q[0:2] = np.arange(10,Q_p_max,Q_p_max//breakpoints_number)
w[0:2] = np.arange(10,W_t_max,W_t_max//breakpoints_number)

Q_pipe_max = 5000
q_pipe = np.zeros((len(pipelines),breakpoints_number))
q[0:len(pipelines)] = np.arange(10,Q_pipe_max,Q_pipe_max//breakpoints_number)

# Variables
model.P_shed = pyo.Var(two_units_idx, domain = pyo.PositiveReals)                         # The shedding amount of load-for reducing demand
model.P_gen = pyo.Var(two_units_idx, domain = pyo.PositiveReals)                          # Expected power output of generating units
model.P_wat = pyo.Var(time_idx, domain = pyo.PositiveReals)                               # Water electricity consumption in bus b-we have only one water bus
model.P_pump = pyo.Var(two_units_idx, domain = pyo.PositiveReals)                       # Power consumption for pump-we have  2 pumps
model.P_total_demand = pyo.Var(time_idx, domain = pyo.PositiveReals)                    # Total power-water demand
model.P_trans_k = pyo.Var(transline_idx, domain = pyo.PositiveReals)                    # Power flow through transmission lines
model.Q_tot = pyo.Var(time_idx, domain = pyo.Reals)                                     # Water flow rate
model.R_flow = pyo.Var(time_idx, domain = pyo.PositiveReals)                            # Reservoir's water inflow rate-We have only one reservoir in simplified model
model.Q_pipe = pyo.Var(Q_pipe_idx, domain = pyo.Reals)                                  # Water flow rate through pipes
model.Q_pump = pyo.Var(two_units_idx, domain = pyo.PositiveReals)                       # Water flow rate through pumps
model.H_tot = pyo.Var(time_idx, domain = pyo.PositiveReals)                             # Pressure heads
model.H_pipe = pyo.Var(H_pipe_idx, domain = pyo.PositiveReals)                          # Pressure heads associated with pipes
model.H_pump = pyo.Var(H_pump_idx, domain = pyo.PositiveReals)                          # Pressure heads associated with pumps
model.H_reservoir = pyo.Var(time_idx, domain = pyo.PositiveReals)                       # Pressure heads associated with reservoir
model.Delta_E = pyo.Var(time_idx, domain = pyo.Reals)                                   # Tanks' inflow/outflow rate difference
model.T_in = pyo.Var(time_idx, domain = pyo.PositiveReals)                              # Water inflow to tanks
model.T_out = pyo.Var(time_idx, domain = pyo.PositiveReals)                             # Water outflow to tanks
model.Vstore_tanks = pyo.Var(time_idx, domain = pyo.PositiveReals)                      # Volume of stored water in tanks
model.W_speed = pyo.Var(two_units_idx, domain = pyo.PositiveReals)                           # Pump's speed
model.X_pipe_bp = pyo.Var(X_pipe_idx, domain = pyo.Reals)                               # Continuous decision variable for pressure head associated with pipes
model.X = pyo.Var(X_idx, domain = pyo.PositiveReals)                              # Continuous decision variable for pressure head associated with pumps
model.Voltage_angle = pyo.Var(Voltage_angle_idx, domain = pyo.Reals)                    # Voltage angle
model.Delta_H = pyo.Var(two_units_idx, domain = pyo.PositiveReals)

# Binary Variables
model.phi_g = pyo.Var(two_units_idx, domain = pyo.Binary)                              # Connection status of generator
model.Nou_k = pyo.Var(transline_idx, domain = pyo.Binary)                              # Connection status of power line
model.Y_p_head = pyo.Var(Y_p_head_idx, domain = pyo.Binary)                            # pressure head breakpoint associated with pipes
model.H_upper = pyo.Var(X_idx, domain = pyo.Binary)
model.H_lower = pyo.Var(X_idx, domain = pyo.Binary)
model.tua = pyo.Var(time_idx, domain = pyo.Binary)                                     # Connection status of pump stations


# Parameters
c_sh_init = [1 for _ in range(24)]
model.tot_elec_demand = pyo.Param(time_idx, within=pyo.Any ,initialize = list(sys_elec_demand.values()))                            # Total electricity demand
model.max_elec_demand = pyo.Param(initialize = 2750)                                      # Maximum elec demand
model.min_elec_demand = pyo.Param(initialize = 1550)                                      # Minimum elec demand
model.transline_reactance = pyo.Param(list(translines.keys()), initialize = [0.0146, 0.2253, 0.1356])  # Reactance of transmission lines
model.max_P_flow = pyo.Param(list(translines.keys()), initialize = [175, 175, 350], within=pyo.Any )                     # Maximum power flow limit of lines
model.max_cap_g = pyo.Param(generating_units, initialize = [152, 152])                                          # Maximum capacity of generating units
model.min_cap_g = pyo.Param(generating_units, initialize = [30.4, 30.4])                                          # Minimum capacity of generating units
model.water_d = pyo.Param(time_idx, within=pyo.Any, initialize = [i+j for i , j  in zip(Water_demand_node4.values(),Water_demand_node7.values())])                                                   # Water demand vector
model.h_Reservoir = pyo.Param(initialize = 0)                                          # Reservoirs' geographical height
model.Vmin_tank = pyo.Param(initialize = 0, within=pyo.Any)                                            # Tanks' maximum volume
model.Vmax_tank = pyo.Param(initialize = 5000, within=pyo.Any)                                            # Tanks' minimum volume
model.min_diff_tank = pyo.Param(initialize = 0)                                        # Minimum charging/discharging difference for tanks
model.max_diff_tank = pyo.Param(initialize = 100)                                        # Maximum charging/discharging difference for tanks
model.P_pump_max = pyo.Param((p for p in range(2)),initialize = [5000,4000])
model.P_pump_min = pyo.Param((p for p in range(2)),initialize = [1000,1000])                       # max/min power consumption for pumps
model.q_pump_bp = pyo.Param(q_p_index,initialize = 3240)                                  # water flow rate of breakpoint for pump
# model.q_pipe_bp = pyo.Param(q_j_index,initialize = 3240)                                  # water flow rate of breakpoint for pipe
model.c_sh = pyo.Param(time_idx, initialize = c_sh_init)                               # The price of shedding load
c_g_init = [0.5 for _ in range(24)]
model.c_g = pyo.Param(time_idx, initialize = c_g_init)                                 # Linear cost coefficients of generating units
model.c_r = pyo.Param(time_idx, initialize = c_g_init)                                 # Linear cost coefficients of generating units


# Define objective function and constrains

In [6]:
# Objective Function
def obj_rule(model):
  return sum(model.c_sh[t]*model.P_shed[sh,t] + model.c_g[t]*model.P_gen[g,t] + model.c_r[t]*model.R_flow[t] for sh in range(2)for g in range(2) for t in time_idx)
  # return sum(sum(sum(model.c_sh[t]*model.P_shed[sh,t] + model.c_g[t]*model.P_gen[g,t] + model.c_r[t]*model.R_flow[t] for sh in range(2))for g in range(2)) for t in time_idx)
model.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize)


######################## WaPS Constrainst ##############################

model.time_set = pyo.Set( initialize=time_idx )
model.translines_set = pyo.Set( initialize=list(translines.keys()) )

def pump_consumption(model, t):
  return sum(model.P_pump[p,t] for p in range(2)) - model.P_wat[t] == 0
model.con1 = pyo.Constraint(model.time_set , rule = pump_consumption)

def integration_water_power(model, t):
  return model.P_total_demand[t] - model.tot_elec_demand[t] - model.P_wat[t] == 0
model.con2 = pyo.Constraint(model.time_set, rule = integration_water_power)

def integration_boundary1(model, t):
  return model.tot_elec_demand[t] + model.P_wat[t] - model.min_elec_demand[t] >= 0
model.con3 = pyo.Constraint(model.time_set ,rule = integration_water_power)

def integration_boundary2(model, t):
  return model.max_elec_demand[t] - model.tot_elec_demand[t] - model.P_wat[t] >= 0
model.con4 = pyo.Constraint(model.time_set ,rule = integration_water_power)

###################### Transmission Line Constraints #####################
def transline_k1(model, t):
  return model.P_trans_k[0,t] == (model.Voltage_angle[0,t]-model.Voltage_angle[1,t])/model.transline_reactance[0]
model.con5 = pyo.Constraint(model.time_set ,rule = transline_k1)

def transline_k2(model, t):
  return model.P_trans_k[1,t] == (model.Voltage_angle[0,t]-model.Voltage_angle[2,t])/model.transline_reactance[1]
model.con6 = pyo.Constraint(model.time_set ,rule = transline_k2)

def transline_k3(model, t):
  return model.P_trans_k[2,t] == (model.Voltage_angle[1,t]-model.Voltage_angle[2,t])/model.transline_reactance[2]
model.con7 = pyo.Constraint(model.time_set ,rule = transline_k3)

def transline_bound1(model,k,t):
  return model.P_trans_k[k,t] >= -model.max_P_flow[k] * model.Nou_k[k,t]
model.con8 = pyo.Constraint(model.translines_set*model.time_set ,rule = transline_bound1)

def transline_bound2(model,k,t):
  return model.P_trans_k[k,t] <= model.max_P_flow[k] * model.Nou_k[k,t]
model.con9 = pyo.Constraint(model.translines_set*model.time_set ,rule = transline_bound2)

############################## Generation Constraints ##########################
def gen_bound1(model,g,t):
  return model.P_gen[g,t] - model.max_cap_g[g]*model.phi_g[g,t] <= 0
model.con10 = pyo.Constraint(generating_units*model.time_set ,rule = gen_bound1)

def gen_bound2(model,g,t):
  return model.P_gen[g,t] - model.min_cap_g[g]*model.phi_g[g,t] >= 0
model.con11 = pyo.Constraint(generating_units*model.time_set ,rule = gen_bound2)

############################# Power Balance Constraints ########################
def power_balance(model,t):
  return sum(model.P_gen[g,t] for g in generating_units) + sum(model.P_shed[i,t] for i in range(2)) - sum(model.P_trans_k[k,t] for k in list(translines.keys())) - model.P_total_demand[t] == 0
model.con12 = pyo.Constraint(model.time_set ,rule = power_balance)

def outage_bound1(model,i,t):
  return model.P_shed[i,t] >= 0
model.con13 = pyo.Constraint(list(range(2))*model.time_set ,rule = outage_bound1)

def outage_bound2(model,i,t):
  return model.P_shed[i,t] <= model.P_total_demand[t]
model.con14 = pyo.Constraint(list(range(2))*model.time_set ,rule = outage_bound2)

############################# Water Flow Constraints ########################

def water_balance(model,t):
  return model.R_flow[t] - model.water_d[t] - model.Q_tot[t] - model.Delta_E[t] == 0
model.con15 = pyo.Constraint(model.time_set ,rule = water_balance)

def Q_bound1(model,t):
  return model.Q_tot[t] + Q_max >= 0
model.con16 = pyo.Constraint(model.time_set ,rule = Q_bound1)

def Q_bound2(model,t):
  return model.Q_tot[t] - Q_max <= 0
model.con17 = pyo.Constraint(model.time_set ,rule = Q_bound2)

def Q_pump_bound(model,p,t):
  return model.Q_pump[p,t] >= 0
model.con18 = pyo.Constraint(list(range(2))*model.time_set ,rule = Q_pump_bound)

def Resv_1(model,t):
  return model.H_reservoir[t] - model.h_Reservoir == 0
model.con19 = pyo.Constraint(model.time_set ,rule = Resv_1)

def Head_bound1(model,t):
  return model.H_tot[t] - H_min >= 0
model.con20 = pyo.Constraint(model.time_set ,rule = Head_bound1)

def Head_bound2(model,t):
  return model.H_tot[t] - H_max <= 0
model.con21 = pyo.Constraint(model.time_set ,rule = Head_bound2)

def Tank_1(model,t):
  return model.Vstore_tanks[t+1] - model.Vstore_tanks[t] - model.Delta_E[t] == 0
model.con22 = pyo.Constraint(list(range(23)) ,rule = Tank_1)

def Tank_flow(model,t):
  return model.Delta_E[t] - model.T_in[t] + model.T_out[t] == 0
model.con23 = pyo.Constraint(model.time_set  ,rule = Tank_flow)

def Tank_flow_diff_1(model,t):
  return model.Delta_E[t] - model.min_diff_tank >= 0
model.con24 = pyo.Constraint(model.time_set  ,rule = Tank_flow_diff_1)

def Tank_flow_diff_2(model,t):
  return model.Delta_E[t] - model.max_diff_tank <= 0
model.con25 = pyo.Constraint(model.time_set  ,rule = Tank_flow_diff_2)

def Tank_2(model,t):
  return model.Vstore_tanks[t] - model.H_tot[t]*(3.14*(20**2)) == 0
model.con26 = pyo.Constraint(model.time_set  ,rule = Tank_2)

def Tank_bound_1(model,t):
  return model.Vstore_tanks[t] - model.Vmin_tank >= 0
model.con27 = pyo.Constraint(model.time_set  ,rule = Tank_bound_1)

def Tank_bound_2(model,t):
  return model.Vstore_tanks[t] - model.Vmax_tank <= 0
model.con28 = pyo.Constraint(model.time_set  ,rule = Tank_bound_2)

def Pump_bound_1(model,p,t):
  return model.P_pump[p,t] - model.P_pump_max[p] <= 0
model.con29 = pyo.Constraint(list(range(2))*model.time_set  ,rule = Pump_bound_1)

def Pump_bound_2(model,p,t):
  return model.P_pump[p,t] - model.P_pump_min[p] >= 0
model.con30 = pyo.Constraint(list(range(2))*model.time_set  ,rule = Pump_bound_2)


# Linearization

## Constraint (13): Piece-wise method

In [7]:
def pipe_24(model,j,t):
  return sum(model.Y_p_head[i,j,t] for i in range(breakpoints_number-1)) == 1
model.con13_1 = pyo.Constraint(list(pipelines.keys())*model.time_set  ,rule = pipe_24)

model.set_brp = pyo.Set(initialize = [(i , j, t) for i in  list(range(1,breakpoints_number)) for j in pipelines.keys() for t in time_idx])

def pipe_25(model,i,j,t):
  return model.X_pipe_bp[i,j,t] <= model.Y_p_head[i,j,t] + model.Y_p_head[i-1,j,t]
model.con13_2 = pyo.Constraint(model.set_brp, rule = pipe_25)

def pipe_26(model,j,t):
  return sum(model.X_pipe_bp[i,j,t] for i in range(breakpoints_number)) == 1
model.con13_3 = pyo.Constraint(list(pipelines.keys())*model.time_set, rule = pipe_26)

def pipe_27(model,j,t):
  return model.X_pipe_bp[breakpoints_number-1,j,t] <= model.Y_p_head[breakpoints_number-2,j,t]
model.con13_4 = pyo.Constraint(list(pipelines.keys())*model.time_set, rule = pipe_27)

def pipe_28(model,j,t):
  return model.X_pipe_bp[1,j,t] <= model.Y_p_head[1,j,t]
model.con13_5 = pyo.Constraint(list(pipelines.keys())*model.time_set, rule = pipe_28)

def pipe_29(model,j,t):
  return model.Q_pipe[j,t] == sum(model.X_pipe_bp[i,j,t]*q_pipe[j,i] for i in range(breakpoints_number))
model.con13_6 = pyo.Constraint(list(pipelines.keys())*model.time_set, rule = pipe_29)

model.set_jnt = pyo.Set(initialize = [(j,n,t) for j in list(pipelines.keys()) for n in range(nodes_number-1) for t in time_idx])
def pipe_30(model,j,n,t):
  return model.H_pipe[j,n,t] - model.H_pipe[j,n+1,t]  == sum(model.X_pipe_bp[i,j,t]*q_pipe[j,i] for i in range(breakpoints_number))
model.con13_7 = pyo.Constraint(model.set_jnt, rule = pipe_30)

## Linearization of constraints (21) & (22) using Triangle method: [paper](https://www.sciencedirect.com/science/article/pii/S0167637709001072)






In [8]:
def X_pump_31(model,p,t):
  return sum(model.X[p,t,u,m] for m in range(breakpoints_number) for u in range(breakpoints_number)) == 1
model.con31 = pyo.Constraint(list(range(2))*model.time_set  ,rule = X_pump_31)

def Xq_pump_32(model,p,t):
  return model.Q_pump[p,t] == sum(model.X[p,t,u,m]*q[p,u] for m in range(breakpoints_number) for u in range(breakpoints_number))
model.con32 = pyo.Constraint(list(range(2))*model.time_set  ,rule = Xq_pump_32)

def Xw_pump_33(model,p,t):
  return model.W_speed[p,t] == sum(model.X[p,t,u,m]*w[p,m] for m in range(breakpoints_number) for u in range(breakpoints_number))
model.con33 = pyo.Constraint(list(range(2))*model.time_set  ,rule = Xw_pump_33)

def Delta_H_34(model,p,t):
  return model.Delta_H[p,t] == sum(w[p,m]*(a[p][0]-a[p][1]*((q[p,u]/w[p,m])**a[p][2]))*model.X[p,t,u,m] for m in range(breakpoints_number) for u in range(breakpoints_number))
model.con34 = pyo.Constraint(list(range(2))*model.time_set  ,rule = Delta_H_34)

def W_pump_35(model,p,t):
  return model.P_pump[p,t] == sum((w[p,m]**3)*(b[p][0]-b[p][1]*(q[p,u]/w[p,m]))*model.X[p,t,u,m] for m in range(breakpoints_number) for u in range(breakpoints_number))
model.con35 = pyo.Constraint(list(range(2))*model.time_set  ,rule = W_pump_35)

def H_binary_36(model,p,t):
  return sum(model.H_upper[p,t,u,m]+model.H_upper[p,t,u,m] for m in range(breakpoints_number) for u in range(breakpoints_number)) == 1
model.con36 = pyo.Constraint(list(range(2))*model.time_set  ,rule = H_binary_36)

def H_binary_37(model,p,t,u,m):
  return model.X[p,t,u,m]<= model.H_upper[p,t,u,m-1] + model.H_upper[p,t,u+1,m] + model.H_upper[p,t,u,m] + model.H_lower[p,t,u-1,m] + model.H_lower[p,t,u,m+1] + model.H_lower[p,t,u,m]
model.con37 = pyo.Constraint(list(range(2))*model.time_set*list(range(1,breakpoints_number-1))*list(range(1,breakpoints_number-1))   ,rule = H_binary_37)

# Solve

In [9]:
import time
opt = pyo.SolverFactory("cbc")
time_solve = time.time()
results = opt.solve(model)
time_end = time.time()

print(results)

  - termination condition: infeasible
  - message from solver: <undefined>



Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 3503
  Number of variables: 6072
  Number of binary variables: 2640
  Number of integer variables: 2640
  Number of nonzeros: 120
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.1
  Wallclock time: 0.15
  Termination condition: infeasible
  Termination message: Model was proven to be infeasible.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
  Error rc: 0
  Time: 0.18799066543579102

